In [1]:
# !sudo apt install libcairo2 libcairo2-dev

In [2]:
from pydantic import BaseModel
from openai import OpenAI
from IPython.display import display, Markdown, HTML
from collections import defaultdict
from PIL import Image as PILImage
from io import BytesIO
import itertools
import json
import uuid
import os
import base64
import re
import cv2
import numpy as np
import random
from typing import Callable
from cairosvg import svg2png
from functools import cmp_to_key
import zipfile
from datetime import datetime

from secret_vars import OPENAI_KEY

client = OpenAI(api_key=OPENAI_KEY)

In [3]:
DISPLAY_SPOILERS = True
GENERATE_THEMES_COUNT = 3
GENERATE_RUNES_COUNT = 20
COMFYUI_PATH = "D:\AI\Flux\ComfyUI"

## Theme generation

In [4]:
class RuneTheme(BaseModel):
    name: str
    description: str

    def __hash__(self):
        return hash((self.name, self.description))

class RuneThemes(BaseModel):
    ideas: list[RuneTheme]

def generate_themes() -> RuneThemes:
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"I'm making a game in which mundane tasks (such as throwing out trash or doing dishes) can be separated each into multiple runes. What overarching theme can I use for coming up with those runes? Reply with a list of {GENERATE_THEMES_COUNT} propositions."},
        ],
        response_format=RuneThemes,
    )
    return completion.choices[0].message.parsed

def display_themes(themes: RuneThemes):
    if not DISPLAY_SPOILERS:
        return
    display(Markdown("\n".join(f"## {theme.name}\n{theme.description}" for theme in themes.ideas)))

themes = generate_themes()
display_themes(themes)

## Nature's Elements
Each rune represents a natural element tied to the task, such as Earth for grounding work like taking out the trash, Water for washing dishes, Air for sweeping, and Fire for tasks that require energy.
## Household Spirits
Each rune embodies a spirit that governs different household tasks, such as the Spirit of Cleanliness for washing, the Spirit of Organization for sorting, the Spirit of Order for throwing things away, and the Spirit of Renewal for chores that refresh the home.
## Celestial Bodies
Runes are inspired by celestial bodies and their properties, like the Moon for tidying and reflecting on the space one lives in, the Sun for energizing tasks like cleaning, the Stars for organizing, and Comets for swiftly discarding unwanted items.

## Rune generation

In [5]:
class Rune(BaseModel):
    name: str
    rune_description: str
    tasks_composed_of: str

    def stripped(self) -> str:
        return re.match("^Rune of(?: the)? (.+?)$", self.name).group(1)

    def encoded(self) -> str:
        return base64.b64encode(self.stripped().encode("ascii")).decode("ascii")

class Runes(BaseModel):
    ideas: list[Rune]

def generate_runes(themes: RuneThemes) -> Runes:
    non_duplicate_runes = defaultdict(list)
    taken_rune_names = set()
    
    i = 0
    while len(taken_rune_names) < GENERATE_RUNES_COUNT:
        theme = themes.ideas[i%len(themes.ideas)]
        
        required_rune_count = round((GENERATE_RUNES_COUNT-len(taken_rune_names))/max(1, len(themes.ideas)-i))
        if i >= len(themes.ideas):
            required_rune_count *= 2
        rune_count = required_rune_count+len(non_duplicate_runes[theme])
        runes = generate_runes_for_theme(theme, rune_count)

        for rune in runes.ideas:
            if rune.stripped() not in taken_rune_names:
                taken_rune_names.add(rune.stripped())
                non_duplicate_runes[theme].append(rune)
            if len(taken_rune_names) >= GENERATE_RUNES_COUNT:
                break

        i += 1
        if i > GENERATE_THEMES_COUNT*3:
            raise Exception("Couldn't generate runes")
    
    for theme, runes in non_duplicate_runes.items():
        display_runes(runes, theme)

    return Runes(ideas = itertools.chain(*non_duplicate_runes.values()))


def generate_runes_for_theme(theme: RuneTheme, count: int) -> Runes:
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"I'm making a game in which mundane tasks (such as throwing out trash or doing dishes) can be separated each into multiple runes. Come up with a list of {count} runes that follow the theme \"{theme.name}\" - {theme.description}. The name of each rune should be \"Rune of ...\". Also, for each rune give a brief descriptions what kind of tasks qualify as being \"composed of\" that rune."},
        ],
        response_format=Runes,
    )
    return completion.choices[0].message.parsed

def display_runes(runes: list[Rune], theme: RuneTheme):
    if not DISPLAY_SPOILERS:
        return
    display(Markdown(f"## {theme.name}:\n" + "\n".join(f" - **{rune.name}**: {rune.rune_description} {rune.tasks_composed_of}" for rune in runes)))

runes = generate_runes(themes)

## Nature's Elements:
 - **Rune of Earth**: Symbolizes stability and grounding. Taking out the trash, organizing storage spaces, tending to a garden, or recycling materials.
 - **Rune of Water**: Represents fluidity and cleansing. Washing dishes, cleaning floors, doing laundry, or watering plants.
 - **Rune of Air**: Embodies movement and lightness. Sweeping, dusting surfaces, ventilating rooms, or organizing papers.
 - **Rune of Fire**: Signifies energy and transformation. Cooking meals, lighting candles, using heat for cleaning, or starting a barbecue.
 - **Rune of Wood**: Reflects growth and nurturing. Planting flowers or trees, caring for houseplants, building furniture, or repairing wooden items.
 - **Rune of Stone**: Embodies durability and permanence. Maintaining outdoor pathways, constructing stone features, or landscaping with rocks.
 - **Rune of Lightning**: Represents speed and clarity. Quickly organizing items, delivering urgent messages, handling electronics with care, or rapidly completing straightforward tasks.

## Household Spirits:
 - **Rune of Cleanliness**: This spirit embodies the essence of cleanliness and hygiene in the household. It ensures that all surfaces are gleaming and free from dust and dirt. Washing dishes, cleaning countertops, sweeping or vacuuming floors, wiping down surfaces, and scrubbing bathrooms.
 - **Rune of Organization**: The spirit that governs the order and arrangement of items within the home. It promotes the ideal placement and sorting of belongings for maximum efficiency. Decluttering spaces, organizing closets or drawers, categorizing books or other items, labeling storage, and arranging furniture for optimal flow.
 - **Rune of Order**: This spirit symbolizes the removal of unnecessary items to create space and clarity in the home. It encourages letting go of things that no longer serve a purpose. Throwing away trash, recycling unusable items, giving away clothes, discarding expired food, and clearing out old paperwork.
 - **Rune of Renewal**: The spirit that breathes fresh life into the household, ensuring that everything is rejuvenated and uplifting. It relates to tasks that refresh the environment. Changing bed linens, freshening up curtains, watering plants, painting walls, and performing seasonal deep cleans.
 - **Rune of Comfort**: This spirit focuses on creating a warm and inviting atmosphere in the household, ensuring that the environment feels cozy and homely. Lighting candles, fluffing pillows, arranging throw blankets, setting up a comfortable seating area, and adjusting lighting for mood.
 - **Rune of Maintenance**: The spirit that oversees the ongoing care and repair of household items, ensuring everything functions at its best. Fixing leaky faucets, replacing light bulbs, sharpening kitchen knives, conducting regular appliance cleaning, and performing seasonal home inspections.

## Celestial Bodies:
 - **Rune of the Sun**: Channel the energy of the Sun to invigorate your cleaning tasks and brighten your space. Dusting surfaces, vacuuming, mopping floors, and any tasks that require a burst of energy.
 - **Rune of the Moon**: Reflect on your surroundings and find peace in tidying up, inspired by the calm light of the Moon. Organizing your closet, rearranging furniture, and gentle decluttering that brings tranquility.
 - **Rune of the Stars**: Use the guidance of the Stars to create order and structure in your living space. Sorting items, rearranging bookshelves, labeling containers, and establishing systems for organization.
 - **Rune of the Comet**: Swiftly remove what no longer serves you, just as a comet trails through space. Throwing out old items, donating clothes, discarding expired food, and any quick elimination of clutter.
 - **Rune of the Nebula**: Embrace the chaos before clarity, like a nebula forming new stars within it. Cleaning out storage spaces, sorting through old papers, and tackling messy areas that require sorting.
 - **Rune of the Galaxy**: Create a harmonious environment by connecting all aspects of your space, inspired by the vastness of galaxies. Coordinating decor, arranging communal areas, organizing shared resources, and fostering an inviting atmosphere.
 - **Rune of the Asteroid**: Break down larger tasks into manageable pieces, like asteroids drifting in space. Dividing chores into smaller steps, planning tasks for the week ahead, and creating a to-do list.

## Generate rune sprites

### Visual concept

In [6]:
class RunePictogram(BaseModel):
    rune: str
    symbol: str

class RunePictograms(BaseModel):
    runes: list[RunePictogram]

class RuneWithPictogram(Rune):
    symbol: str

def generate_runes_pictograms(runes: Runes) -> list[RuneWithPictogram]:
    rune_names = "\n".join([rune.name for rune in runes.ideas])
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"What rune-like symbol/pictogram would best fit the following runes? For each rune, reply with a single answer containing max three words. Try to make each rune recognizable and distinct from the other ones. Make all your ideas \"fit\" with one another. Prefer simple symbols over intricate ones. Remember that those are supposed to be font-like characters for runes, not entire pictures.\n\n{rune_names}"},
        ],
        response_format=RunePictograms,
    )
    pictograms = completion.choices[0].message.parsed
    
    pictogram_map = {p.rune: p.symbol for p in pictograms.runes}
    return [
        RuneWithPictogram(**rune.dict(), symbol = pictogram_map.get(rune.name, rune.name))
        for rune in runes.ideas
    ]

def display_runes_pictograms(runes: list[RuneWithPictogram]):
    if not DISPLAY_SPOILERS:
        return
    display(Markdown("\n".join(f" - **{rune.name}**: {rune.symbol}" for rune in runes)))

rune_pictograms = generate_runes_pictograms(runes)
display_runes_pictograms(rune_pictograms)

 - **Rune of Earth**: Triangle Base
 - **Rune of Water**: Wavy Line
 - **Rune of Air**: Curved Line
 - **Rune of Fire**: Flame Shape
 - **Rune of Wood**: Tree Silhouette
 - **Rune of Stone**: Circle Rock
 - **Rune of Lightning**: Zigzag Bolt
 - **Rune of Cleanliness**: Water Drop
 - **Rune of Organization**: Stacked Lines
 - **Rune of Order**: Grid Pattern
 - **Rune of Renewal**: Circular Arrow
 - **Rune of Comfort**: Soft Cushion
 - **Rune of Maintenance**: Wrench Icon
 - **Rune of the Sun**: Radiating Circle
 - **Rune of the Moon**: Crescent Shape
 - **Rune of the Stars**: Five-Point Star
 - **Rune of the Comet**: Streaking Line
 - **Rune of the Nebula**: Swirling Spiral
 - **Rune of the Galaxy**: Spiral Galaxy
 - **Rune of the Asteroid**: Irregular Shape

### Initial sprite generation

In [7]:
def generate_rune_sprites(runes: list[RuneWithPictogram]):
    path_here = os.popen('powershell.exe -Command "(Get-Item .).FullName"').read().replace("\n", "")
    uid = uuid.uuid4()
    runes_str = base64.b64encode(json.dumps([rune.symbol for rune in runes]).encode("ascii")).decode("ascii")

    command = f'cd {COMFYUI_PATH}; python "{path_here}\image_gen_runes.py" --uuid "{uid}" --runes "{runes_str}"'
    full_command = f'powershell.exe -Command {json.dumps(command)}'
    os.system(full_command)

    commands = []
    for rune in runes:
        for i in range(1, 5):
            command = f'Move-Item -Path "{COMFYUI_PATH}\\output\\rune_{uid}_{rune.symbol}_0000{i}_.png" -Destination "{path_here}\\img\\{rune.encoded()}_{i}.png" -Force'
            commands.append(command)
    os.system(f'powershell.exe -Command {json.dumps("; ".join(commands))}')

generate_rune_sprites(rune_pictograms)

Total VRAM 12282 MB, total RAM 32609 MB
pytorch version: 2.5.0+cu124
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.5.0+cu124)
    Python  3.10.11 (you have 3.10.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
C:\Users\Aurif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xformers\ops\swiglu_op.py:107: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(cls, ctx, x, w1, b1, w2, b2, w3, b3):
C:\Users\Aurif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xformers\ops\swiglu_op.py:

ComfyUI found: D:\AI\Flux\ComfyUI
'D:\AI\Flux\ComfyUI' added to sys.path
Could not find the extra_model_paths config file.
ComfyUI found: D:\AI\Flux\ComfyUI
'D:\AI\Flux\ComfyUI' added to sys.path


100%|##########| 15/15 [00:15<00:00,  1.06s/it]
Requested to load AutoencodingEngine
Loading 1 new model
loaded completely 0.0 159.87335777282715 True
Requested to load FluxClipModel_
Loading 1 new model
loaded completely 0.0 4777.53759765625 True
loaded partially 9545.67451171875 9545.5224609375 0
100%|##########| 15/15 [00:13<00:00,  1.13it/s]
Requested to load AutoencodingEngine
Loading 1 new model
loaded completely 0.0 159.87335777282715 True
Requested to load FluxClipModel_
Loading 1 new model
loaded completely 0.0 4777.53759765625 True
loaded partially 9545.67451171875 9545.5224609375 0
100%|##########| 15/15 [00:12<00:00,  1.19it/s]
Requested to load AutoencodingEngine
Loading 1 new model
loaded completely 0.0 159.87335777282715 True
Requested to load FluxClipModel_
Loading 1 new model
loaded completely 0.0 4777.53759765625 True
loaded partially 9545.67451171875 9545.5224609375 0
100%|##########| 15/15 [00:12<00:00,  1.20it/s]
Requested to load AutoencodingEngine
Loading 1 new m

In [8]:
def display_raw_rune_sprites(runes: list[RuneWithPictogram], *, suffix: str = ".png"):
    if not DISPLAY_SPOILERS:
        return
    
    full_code = ""
    for rune in runes:
        img_code = "\n".join([f'<img src="img/{rune.encoded()}_{i}{suffix}?{random.randint(0, 1e7)}" style="width: 128px" />' for i in range(1, 5)])
        html_code = f"""
            <div>
                <div style="display: flex; justify-content: space-around;">
                    <span style="font-size: 1.6em; font-weight: 600">{rune.name}</span>
                </div>
                <div style="display: flex; justify-content: space-around;">
                    {img_code}
                </div>
            </div>
        """
        full_code += html_code

    full_code = f"""
        <div style="display: flex; flex-direction: row; flex-wrap: wrap; justify-content: space-around;">
            {full_code}
        </div>
    """
    display(HTML(full_code))
display_raw_rune_sprites(rune_pictograms)

### Sprite post-processing

In [9]:
def process_all_images(runes: list[RuneWithPictogram], func: Callable):
    for rune in runes:
        for i in range(1, 5):
            try:
                func(f'{rune.encoded()}_{i}')
            except Exception as e:
                print(f'{rune.encoded()}_{i}')
                raise e

def post_process_sprite(path: str):
    TARGET_SIZE = 128

    def find_segments(image: np.ndarray) -> list[np.ndarray]:
        gray = cv2.cvtColor(cv2.bitwise_not(image), cv2.COLOR_BGRA2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        close_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (6,6))
        close = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, close_kernel, iterations=2)

        dilate_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        dilate = cv2.dilate(close, dilate_kernel, iterations=1)

        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]

        return cnts

    def retain_only_largest_segment(image: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
        segments_to_remove = []

        max_cnt = (0, None)
        for c in find_segments(image):
            area = cv2.contourArea(c)
            if area > max_cnt[0]:
                if max_cnt[1] is not None:
                    segments_to_remove.append(max_cnt[1])
                max_cnt = (area, c)
            else:
                segments_to_remove.append(c)

        for c in segments_to_remove:
            x,y,w,h = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 0, 0), -1)

        return image, max_cnt[1]

    def crop_and_resize(image: np.ndarray, rect: np.ndarray) -> np.ndarray:
        x,y,w,h = cv2.boundingRect(rect)
        image = image[y:y+h, x:x+w]

        if w > h:
            resized_image = cv2.resize(image, (TARGET_SIZE, int(h*(TARGET_SIZE/w))))
        else:
            resized_image = cv2.resize(image, (int(w*(TARGET_SIZE/h)), TARGET_SIZE))

        top_padding = (TARGET_SIZE - resized_image.shape[0]) // 2
        bottom_padding = TARGET_SIZE - resized_image.shape[0] - top_padding
        left_padding = (TARGET_SIZE - resized_image.shape[1]) // 2
        right_padding = TARGET_SIZE - resized_image.shape[1] - left_padding

        padded_image = cv2.copyMakeBorder(
            resized_image,
            top_padding,
            bottom_padding,
            left_padding,
            right_padding,
            borderType=cv2.BORDER_CONSTANT,
            value=[0, 0, 0, 0]
        )

        return padded_image

    def normalize_colors(image: np.ndarray) -> np.ndarray:
        original_size = image.shape
        image = cv2.resize(image, (original_size[0]*4, original_size[1]*4))

        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
        mask = gray_image < 96
        image[mask] = [255, 255, 255, 255]
        image[~mask] = [0, 0, 0, 0]

        image = cv2.resize(image, (original_size[0], original_size[1]))
        return image
    
    def remove_small_details(image: np.ndarray) -> np.ndarray:
        image_post = cv2.copyMakeBorder(
            image,
            4, 4, 4, 4,
            borderType=cv2.BORDER_CONSTANT,
            value=[0, 0, 0, 0]
        )

        _, binary = cv2.threshold(cv2.cvtColor(image_post, cv2.COLOR_BGRA2GRAY), 128, 255, cv2.THRESH_BINARY)
        kernel = np.ones((2, 2), np.uint8)
        image_post = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

        _, binary = cv2.threshold(cv2.bitwise_not(image_post), 128, 255, cv2.THRESH_BINARY)
        kernel = np.ones((2, 2), np.uint8)
        image_post = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

        image_post = image_post[4:-4, 4:-4]

        mask = image_post < 96
        if np.count_nonzero(mask) == 0:
            return image
        
        image[mask] = [255, 255, 255, 255]
        image[~mask] = [0, 0, 0, 0]
        return image
    
    image = cv2.imread(f'img/{path}.png')
    image = normalize_colors(image)
    image, max_segment = retain_only_largest_segment(image)
    image = crop_and_resize(image, max_segment)
    image = remove_small_details(image)
    cv2.imwrite(f'img/{path}_post.png', image)

process_all_images(rune_pictograms, post_process_sprite)
display_raw_rune_sprites(rune_pictograms, suffix="_post.png")

### Sprite vectorization and stylization

In [10]:
def vectorize_sprite(path: str):
    image = cv2.imread(f"img/{path}_post.png", cv2.IMREAD_GRAYSCALE)
    _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)

    contours, hierarchy = cv2.findContours(binary_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    def contour_to_path(contour: np.ndarray, style: str) -> str:
        contour = cv2.approxPolyDP(contour, 5.0, True)
        path = '<path d="M '
        for point in contour:
            x, y = point[0]
            path += f'{x},{y} '
        path += f'Z" {style}/>\n'
        return path

    with open(f"img/{path}.svg", 'w') as svg_file:
        svg_file.write('<?xml version="1.0" standalone="no"?>\n')
        svg_file.write('<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 20010904//EN"\n')
        svg_file.write(' "http://www.w3.org/TR/2001/REC-SVG-20010904/DTD/svg10.dtd">\n')
        svg_file.write(f'<svg xmlns="http://www.w3.org/2000/svg" version="1.1" viewBox="0 0 {image.shape[0]} {image.shape[1]}">\n')
        
        # Inner elements
        svg_file.write(f'<mask id="cutout-mask" x="0" y="0" width="{image.shape[0]}" height="{image.shape[1]}" maskUnits="userSpaceOnUse" maskContentUnits="userSpaceOnUse">\n')
        svg_file.write(f'\t<rect x="0" y="0" width="{image.shape[0]}" height="{image.shape[1]}" fill="white" />\n')

        for i, contour in enumerate(contours):
            if hierarchy[0][i][3] == -1:
                continue # Outer contour
            elif hierarchy[0][i][3] == 0:
                fill = "black" # Holes in shape
            elif hierarchy[0][hierarchy[0][i][3]][3] == 0:
                fill = "white" # Inner elements
            else:
                continue # Overly complex, ignoring
            
            svg_file.write('\t'+contour_to_path(contour, f' stroke="white" fill="{fill}"'))

        svg_file.write('</mask>\n')

        # Outer contour
        for i, contour in enumerate(contours):
            if hierarchy[0][i][3] != -1:
                continue
            svg_file.write(contour_to_path(contour, ' stroke="white" fill="white" mask="url(#cutout-mask)"'))
        
        svg_file.write('</svg>\n')

process_all_images(rune_pictograms, vectorize_sprite)
display_raw_rune_sprites(rune_pictograms, suffix=".svg")

### Sprite rating

In [11]:
class SpriteRating(BaseModel):
    simplicity: int
    rune_like: int
    aesthetic: int
    resemblance: int
    contains_text: bool

class SpriteRatings(BaseModel):
    top_left: SpriteRating
    top_right: SpriteRating
    bottom_left: SpriteRating
    bottom_right: SpriteRating

class RuneWithSpriteRatings(RuneWithPictogram):
    ratings: SpriteRatings

def select_sprite(rune: RuneWithPictogram) -> RuneWithSpriteRatings:
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"The attached image is a 2x2 grid of symbols, each occuppying one quadrant. Rate each of the symbols from 1 to 7 in each of the following categories:\n- simplicity of the design, with 7 being a very simple design, and 1 being overly complex\n- similarity to celtic runes or egyptian hieroglyphs, with 7 meaning it fits well with them, and 1 meaning it doesn't fit at all\n - the aesthetic design of the symbol, with 7 being very pretty and well designed one, and 1 being a very poorly designed one\n - resemblance of {rune.stripped()}, with 7 meaning it can be unmistakenly interpreted as {rune.stripped()}, and 1 meaning it has no correlation with {rune.stripped()}\n- identify if that quadrant of the image contains any text",                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url":  f"data:image/png;base64,{prepare_sprite_grid(rune)}",
                            "detail": "high"
                        },
                    },
                ],
            }
        ],
        response_format=SpriteRatings
    )

    rating = response.choices[0].message.parsed
    rune = RuneWithSpriteRatings(**rune.dict(), ratings=rating)
    return rune

def read_svg(path: str) -> PILImage:
    with open(path, 'r') as f:
        png = svg2png(bytestring=f.read(), output_width=128, output_height=128)
    return PILImage.open(BytesIO(png))
    
def prepare_sprite_grid(rune: RuneWithPictogram) -> str:
    images = [read_svg(f"img/{rune.encoded()}_{i}.svg") for i in range(1, 5)]

    grid_image = PILImage.new("RGBA", (256, 256))
    grid_image.paste(images[0], (0, 0))        # Top-left
    grid_image.paste(images[1], (128, 0))      # Top-right
    grid_image.paste(images[2], (0, 128))      # Bottom-left
    grid_image.paste(images[3], (128, 128))    # Bottom-right

    return encode_image(grid_image)

def encode_image(img: PILImage) -> str:
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


runes_with_ratings = [select_sprite(rune) for rune in rune_pictograms]

In [12]:
class SpriteExtendedRating(SpriteRating):
    paths: int
    vertices: int
    coverage: float

class RuneWithSprite(RuneWithSpriteRatings):
    sprite_path: str

def rank_sprites(rune: RuneWithSpriteRatings) -> RuneWithSprite:
    def make_ranking_map(values: list[int]) -> dict[int, int]:
        ranks = np.array(values).argsort().argsort()
        return {
            v: ranks[i]
            for i, v in enumerate(values)
        }
    
    def extend_rating(rating: SpriteRating, i: int) -> SpriteExtendedRating:
        with open(f"img/{rune.encoded()}_{i}.svg", 'r') as f:
            svg = f.read()
        paths = svg.count("path")
        vertices = svg.count(",")

        img = read_svg(f"img/{rune.encoded()}_{i}.svg")
        img_data = img.convert("L").load()
        white_pixels = sum(1 for x in range(img.size[0]) for y in range(img.size[1]) if img_data[x, y] == 255)
        coverage = white_pixels / img.size[0] / img.size[1]
        coverage = round(round(coverage/5, 2)*5, 3)
        
        return SpriteExtendedRating(
            paths=-paths,
            vertices=-vertices,
            coverage=coverage,
            **rating.dict()
        )

    ratings = [(1, rune.ratings.top_left), (2, rune.ratings.top_right), (3, rune.ratings.bottom_left), (4, rune.ratings.bottom_right)]
    ratings = [(i, extend_rating(r, i)) for i, r in ratings]
    rating_keys = ratings[0][1].dict().keys()
    rankings = {
        k: make_ranking_map([getattr(r[1], k) for r in ratings])
        for k in rating_keys
    }
    
    ratio = {
        'resemblance': 1.5
    }

    def score(r: SpriteRating) -> int:
        return np.sum([rankings[k][getattr(r, k)]*ratio.get(k, 1) for k in rating_keys])
    
    def compare_ratings(r1: tuple[int, SpriteExtendedRating], r2: tuple[int, SpriteExtendedRating]) -> int:
        r1 = r1[1]
        r2 = r2[1]

        if r1.contains_text and not r2.contains_text:
            return -1
        if r2.contains_text and not r1.contains_text:
            return 1
        

        return score(r1) - score(r2)
    
    ratings.sort(key=cmp_to_key(compare_ratings), reverse=True)

    if DISPLAY_SPOILERS:
        def serialize_ratings(r: SpriteRating) -> str:
            stats = "\n".join([f"{k}: {abs(v)} ({rankings[k][v]})" for k, v in r.dict().items()])
            stats += f"\n\nTOTAL: {score(r)}"
            return stats
        
        img_code = "".join([
            f"""
            <div style="display: flex; flex-direction: row;">
                <img src="img/{rune.encoded()}_{r[0]}.svg?{random.randint(0, 1e7)}" style="width: 196px" />
                <div style="white-space: pre; margin-left: 8px; font-size: 16px">{serialize_ratings(r[1])}</div>
            </div>
            """
            for r in ratings
        ])
        full_code = f"""
            <div style="display: flex; justify-content: space-around;">
                <span style="font-size: 1.6em; font-weight: 600">{rune.name}</span>
            </div>
            <div style="display: flex; flex-direction: row; flex-wrap: wrap; justify-content: space-around;">
                {img_code}
            </div>
        """
        display(HTML(full_code))

    return RuneWithSprite(
        sprite_path=f"img/{rune.encoded()}_{ratings[0][0]}.svg",
        **rune.dict()
    )

runes_with_sprites = [rank_sprites(rune) for rune in runes_with_ratings]

In [13]:
def display_final_rune_sprites(runes: list[RuneWithSprite]):
    if not DISPLAY_SPOILERS:
        return
    
    full_code = ""
    for rune in runes:
        full_code += f'<img src="{rune.sprite_path}?{random.randint(0, 1e7)}" style="width: 64px; margin: 12px" />'

    full_code = f"""
        <div style="display: flex; flex-direction: row; flex-wrap: wrap; justify-content: space-around; width: 30%; margin-left: 35%">
            {full_code}
        </div>
    """
    display(HTML(full_code))

display_final_rune_sprites(runes_with_sprites)

## Export

In [23]:
class RuneToExport(BaseModel):
    name: str
    description: str
    task_description: str
    sprite_svg: str

def encode(value: str) -> str:
    return base64.b64encode(value.encode("ascii")).decode("ascii")

def export_runes(runes: list[RuneWithSprite]) -> str:
    exported_runes: list[RuneToExport] = []
    for rune in runes:
        with open(rune.sprite_path, 'r') as f:
            svg = f.read()
        exported_runes.append(
            RuneToExport(
                name=encode(rune.name),
                description=encode(rune.rune_description),
                task_description=encode(rune.tasks_composed_of),
                sprite_svg=svg
            )
        )
    return json.dumps([r.dict() for r in exported_runes])
        
def export_all():
    runes = export_runes(runes_with_sprites)
    meta = {
        "version": "0.1.0",
        "signature": "Handcrafted for Ari"
    }

    with zipfile.ZipFile(f"dist/{meta['version'].replace('.', '-')}___{datetime.now().strftime('%Y-%m-%d_%H-%M')}.zip", "w") as zip_file:
        zip_file.writestr("definitions/runes.json", json.dumps(runes))
        zip_file.writestr("meta.json", json.dumps(meta))

export_all()